In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import csv
import os
import time
import requests
from bs4 import BeautifulSoup

In [4]:
url =' https://www.buyrentkenya.com/houses-for-sale'
# set up the webscrapping agent for chrome
agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
# headers dictionary to send with the GET request
HEADERS = {'User-Agent': agent, 'Accept-Language': 'en-US, en'}

In [5]:
# get the data
data = requests.get(url, headers=HEADERS)

In [6]:
# creating a soup object
soup = BeautifulSoup(data.text, 'html.parser')

In [7]:
# Finding houses data in the div
# because class is a reserved keyword in Python
results = soup.find_all('div', class_='listing_card')

In [8]:
# lists to store data in
titles = []
prices = []
descriptions = []
locations = []
no_of_bedrooms = []
no_of_bathrooms = []


In [9]:
# find house details
houses = soup.find_all('div', class_='listing-card')

In [10]:
# loop through first page houses
for page in range(1,188):

    for house in houses:
        title = house.find('span', class_='relative top-[2px] hidden md:inline').text.strip()
        price = house.find('p', class_='text-xl font-bold leading-7 text-grey-900').text.strip()
        description = house.find('a',class_='block truncate text-grey-500 no-underline').text.strip()
        location = house.find('p', class_='ml-1 truncate text-sm font-normal capitalize text-grey-650').text.strip()
        no_of_bedroom = house.find_all('span', class_='whitespace-nowrap font-normal')[0].text.strip()
        no_of_bathroom = house.find_all('span', class_='whitespace-nowrap font-normal')[1].text.strip()

        titles.append(title)
        locations.append(location)
        descriptions.append(description)
        prices.append(price)
        no_of_bedrooms.append(no_of_bedroom)
        no_of_bathrooms.append(no_of_bathroom)
        

In [11]:
# check whether all the data was scraped successfully
len(titles) == len(locations) == len(descriptions) == len(prices) == len(no_of_bedrooms) == len(no_of_bathrooms)

True

In [12]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [17]:
# create a dataset and store the scraped data
import pandas as pd
df = pd.DataFrame(
    {'Title': titles,
     'Location': locations,
     'Description': descriptions,
     'Price': prices,
     'No_of_bedrooms': no_of_bedrooms,
     'No_of_bathrooms': no_of_bathrooms}
)

In [18]:
df.head()

,Title,Location,Description,Price,No_of_bedrooms,No_of_bathrooms
0,4 Bed Villa with En Suite at 00620,"64911, 00620, Thika Road",The Oint Country Homes,"KSh 24,800,000",4,4
1,5 Bed House with En Suite at Karen Road,"Karen Road, Karen","5 Bedrooms Villas For Sale in Karen, Aria Park","KSh 140,000,000",5,6
2,4 Bed Townhouse with En Suite at Gikambura,"GIKAMBURA, Kikuyu Town, Kikuyu","4 BEDROOM TOWNHOUSE IN KIKUYU,@11.5M","KSh 11,500,000",4,4
3,4 Bed Townhouse with En Suite in Lavington,Lavington,Town House For Sale -Lavington along Njumbi Ro...,"KSh 80,000,000",4,4
4,5 Bed House with En Suite at Runda,"Runda, Westlands",1.629 ACRES FOR SALE IN RUNDA NAIROBI KENYA,"KSh 585,000,000",5,5


In [30]:
df.shape

(3366, 6)

In [34]:
df.to_csv('rents.csv', index=False, encoding='utf-8')

In [32]:
# find duplicates
df_clean = df.drop_duplicates()

In [33]:
df_clean.shape

(18, 6)

In [20]:
df.shape

(3366, 6)

EDA

In [ ]:
# Duplicates(find and drop)

In [75]:
# Find nulls
df.isna().sum()

Title                0
Location             0
Description          0
Price                0
No_of_bedrooms       0
No_of_bathrooms    187
dtype: int64

In [70]:
# Remove Ksh prefix, commas and convert to int
# df['Price'] = df['Price'].astype('string')
df['Price'] = pd.to_numeric(df['Price'].str.replace('KSh', '').str.replace('Ksh', '').str.replace(',', '').str.strip())

In [58]:
df['No_of_bathrooms'] = pd.to_numeric(df['No_of_bathrooms'])
df['No_of_bedrooms'] = pd.to_numeric(df['No_of_bedrooms'])

In [74]:
# Finding null values in no. of bathrooms column
df['No_of_bathrooms'].isna().sum()


187

In [76]:
# Replace NA in No of bathrooms with 0
df['No_of_bathrooms'] = df['No_of_bathrooms'].fillna(0)

In [79]:
# Convert type to int
df['No_of_bathrooms'] = df['No_of_bathrooms'].astype(int)

In [82]:
# find data types
df.dtypes

Title              object
Location           object
Description        object
Price               Int64
No_of_bedrooms      int64
No_of_bathrooms     int32
dtype: object

In [87]:
df_clean = df

In [96]:
df['Location'] = df['Location'].str.split(',').str[-1].str.strip()

In [102]:
df.head()

,Title,Location,Description,Price,No_of_bedrooms,No_of_bathrooms
0,4 Bed Villa with En Suite at 00620,Thika Road,The Oint Country Homes,24800000,4,4
1,5 Bed House with En Suite at Karen Road,Karen,"5 Bedrooms Villas For Sale in Karen, Aria Park",140000000,5,6
2,4 Bed Townhouse with En Suite at Gikambura,Kikuyu,"4 BEDROOM TOWNHOUSE IN KIKUYU,@11.5M",11500000,4,4
3,4 Bed Townhouse with En Suite in Lavington,Lavington,Town House For Sale -Lavington along Njumbi Ro...,80000000,4,4
4,5 Bed House with En Suite at Runda,Westlands,1.629 ACRES FOR SALE IN RUNDA NAIROBI KENYA,585000000,5,5


In [104]:
df.duplicated().sum()

3348

In [110]:
print(df['No_of_bedrooms'].corr(df['Price']))

0.5425834538790998


In [108]:
print(df['Price'].corr(df['No_of_bathrooms']))

0.39506938563461763


In [109]:
print(df['No_of_bedrooms'].corr(df['No_of_bathrooms']))

0.8353497940087828
